In [ ]:
import os
assert os.environ[
    'COLAB_TPU_ADDR'], 'Make sure to select TPU from Edit > Notebook settings > Hardware accelerator'

In [ ]:
VERSION = "1.5"  #@param ["1.5" , "20200325", "nightly"]
!curl https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py
!python pytorch-xla-env-setup.py --version $VERSION

In [ ]:
import os

! git clone https://github.com/fengredrum/cnn-xla.git
os.chdir('cnn-xla')

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import torch_xla
import torch_xla.distributed.xla_multiprocessing as xmp
import torch_xla.utils.utils as xu

from utils import load_data_cifar_10, train_model_xla
from models import alexnet, resnet18, densenet18

In [ ]:
model_name = "alexnet"  #@param ["alexnet", "resnet", "densenet"]

activation = "mish"  #@param ["relu", "swish", "mish"]

batch_size = 256  #@param {type:"integer"}

lr = 0.01  #@param {type:"number"}

num_epochs = 20  #@param {type:"integer"}

In [ ]:
# Define Parameters
FLAGS = {}
FLAGS['batch_size'] = batch_size
FLAGS['lr'] = lr
FLAGS['num_epochs'] = num_epochs
FLAGS['num_cores'] = 8

In [ ]:
# Start training processes
def _mp_fn(rank, flags):
    global FLAGS
    FLAGS = flags
    torch.set_default_tensor_type('torch.FloatTensor')
    if model_name == 'alexnet':
        net = alexnet(activation=activation)
    elif model_name == 'resnet':
        net = resnet18([2, 2, 2, 2], activation=activation)
    elif model_name == 'densenet':
        net = densenet18([4, 4, 4, 4], activation=activation)
    else:
        raise NotImplementedError
    accuracy, data, pred, target = train_model_xla(net, FLAGS['batch_size'],
                                                   FLAGS['lr'],
                                                   FLAGS['num_epochs'])


xmp.spawn(_mp_fn,
          args=(FLAGS, ),
          nprocs=FLAGS['num_cores'],
          start_method='fork')